In [12]:
import random
random.seed(0)

def load_data(filename):
    lines = open(filename).read().strip().split('\n')
    lines = [line.strip() for line in lines]
    lines = [line for line in lines if line != ""]
    random.shuffle(lines)
    return lines

content = load_data("content.txt")
unwanted = load_data("unwanted.txt")
print('\n'.join(content[:2]))
print('\n'.join(unwanted[:2]))

Making statements based on opinion; back them up with references or personal experience.
Disney's Frozen represents a landmark for the animation giant due not only to its immense popularity but also its introduction of the studio's first disabled princess. In order to make Elsa's story possible, the animators use a combination of narrative devices including the introduction of a second princess, whose story fulfills the audience's expectation for a traditional "princess journey," their patented aesthetic of cuteness, and the encoding of disability as fantasy. Although Elsa's disability is encoded as a magical ice power, the language the film uses to talk about her condition maps on to the experiences of people with physical, mental, and intellectual disabilities in recognizable ways. Meanwhile, her status as a much-beloved princess figure allows the animators at Disney to position disability as a universal experience and in turn to create empathy for PWDs both on and off screen.
<<Comp

In [13]:
import re
import numpy as np

key_chars = """ !"$&'()*+,-./0:;<=>?A[\\]^_a{}"""

def vectorize(s):
    s = re.sub(r"[a-z]", 'a', s)
    s = re.sub(r"[A-Z]", 'A', s)
    s = re.sub(r"[0-9]", '0', s)
    s = re.sub(r"\s", ' ', s)
    counts = dict(zip(key_chars, [0]*len(key_chars)))
    for c in s:
        if c in counts:
            counts[c] += 1
    return np.array([counts[c] for c in key_chars], dtype=np.float32)

print(vectorize(''.join(content)))
print(vectorize(''.join(unwanted)))


[8.45160e+04 2.12000e+02 2.06700e+03 1.00000e+02 1.80000e+01 7.62000e+02
 5.91000e+02 5.90000e+02 2.00000e+00 2.80000e+01 6.65100e+03 1.04500e+03
 4.28500e+03 4.90000e+01 3.35200e+03 1.45000e+02 4.86000e+02 3.00000e+00
 5.70000e+01 1.00000e+00 2.99000e+02 1.23640e+04 2.20000e+02 4.19000e+02
 2.20000e+02 7.90000e+01 2.42000e+02 3.85363e+05 1.42000e+02 1.42000e+02]
[1.3062e+04 1.4000e+01 4.3900e+02 1.4100e+02 7.2000e+01 2.7600e+02
 4.2800e+02 4.2800e+02 4.0000e+01 9.9000e+01 1.2270e+03 2.7800e+02
 1.3190e+03 5.0000e+01 3.9640e+03 2.2000e+02 7.5000e+01 5.0000e+01
 1.2200e+02 7.6000e+01 1.3000e+01 8.2940e+03 4.9000e+01 4.7600e+02
 4.9000e+01 2.1800e+02 1.4900e+02 4.7826e+04 2.8800e+02 2.8700e+02]


In [31]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):

    def __init__(self):
        super().__init__()

    def __len__(self):
        return len(content) + len(unwanted)

    def __getitem__(self, i):
        if i < len(content):
            return vectorize(content[i]), \
                np.array([1.], dtype=np.float32)
        return vectorize(unwanted[i-len(content)]), \
            np.array([0.], dtype=np.float32)

dataloader = DataLoader(MyDataset(), batch_size=32, shuffle=True)
dataloader = [d for d in dataloader]
test_data = dataloader[0]
train_data = dataloader[1:]
print(test_data[0].shape[1])

class Net(nn.Module):

    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Linear(test_data[0].shape[1], 1, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x)

net = Net()
print(net)

loss = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters())

for epoch in range(500):
    for x, y in train_data:
        net.zero_grad()
        err = loss(net(x), y)
        err.backward()
        optimizer.step()
    predicted = net(test_data[0])
    actual = test_data[1]
    err = loss(predicted, actual).detach().numpy()
    accuracy = np.mean((torch.round(predicted) == actual).detach().numpy())
    if (epoch+1) % 50 == 0:
        print(f"test ({epoch+1}):", np.mean(err), accuracy)

30
Net(
  (main): Sequential(
    (0): Linear(in_features=30, out_features=1, bias=True)
    (1): Sigmoid()
  )
)
test (50): 0.14024344 0.96875
test (100): 0.12551524 0.96875
test (150): 0.121369 0.96875
test (200): 0.11962508 0.96875
test (250): 0.118796125 0.96875
test (300): 0.118389495 0.96875
test (350): 0.11818839 0.96875
test (400): 0.11808909 0.96875
test (450): 0.11804059 0.96875
test (500): 0.11801704 0.96875


In [49]:
ws = []
for param in net.parameters():
    weights = param.data
    ws.append(weights.numpy())
weights = [round(w, 4) for w in ws[0][0].tolist()]
weights = dict(zip(key_chars, weights))
bias = round(ws[1][0], 4)
bias = -1.2
print(weights)
print(bias)

def predict(s):
    s0 = s
    s = re.sub(r"[a-z]", 'a', s)
    s = re.sub(r"[A-Z]", 'A', s)
    s = re.sub(r"[0-9]", '0', s)
    s = re.sub(r"\s", ' ', s)
    total = bias
    for c in s:
        if c in weights:
            total += weights[c]
    print(round(total, 4), s0)

predict("MOANA")
predict("Moana")
predict("I am Moana of Motunui.")
predict("Do you want to build a snowman?")
predict("I am a sussy programmer writing sussy code.")
predict("What's wrong with you?")
predict("Creates a criterion that measures the Binary Cross Entropy between the target and the input probabilities:")
predict("Table of Contents")
predict("https://harry7557558.github.io")
predict("1.1 4th-Order Partial Differential Equations")
predict("4th-Order Partial Differential Equations")
predict("4th-order partial differential equations")
predict("From Wikipedia, the Free Encyclopedia.")
predict("From Wikipedia, the free encyclopedia.")
predict("Harry Chen - Updated 12/30/2022")
predict("Note: This tool is recently under active development.")
predict("initialize WebGL: load and compile shader, initialize buffers")
predict("tell WebGL how to pull out the positions from the position buffer into the vertexPosition")
predict("## A list/backup of my saved Desmos graphs")
predict("spiral, logarithmic, interior, beach, nautilus, seashell, conch • 2021/09/12 • public+api")
predict("Debugging an SDF, try to use a previous volume rendering shader to visualize its discontinuity.")
predict("[6] A. Witkin, \"Physically based modeling: Particle system dynamics,\" https://graphics.pixar.com/pbm2001/pdf/notesc.pdf, 2001.")
predict("Kierkegaard, Søren Aabye. Source of Søren Kierkegaard Quote. College of Liberal Arts & Sciences, The University of Iowa, https://homepage.math.uiowa.edu/~jorgen/kierkegaardquotesource.html.")
predict("T=2\pi\sqrt{\\frac{L}{g}}\left(1+\\frac{1}{16}\\theta^2+\mathrm{O}\left(\\theta^4\\right)\\right)")
predict("The Preconditioned Conjugate Gradient Method")


{' ': -0.0085, '!': 1.8875, '"': 0.9782, '$': -0.1644, '&': 0.5831, "'": -0.6959, '(': -0.0929, ')': -0.0961, '*': 0.428, '+': -0.2615, ',': 0.0972, '-': 0.4882, '.': 0.3976, '/': 0.0095, '0': -0.3535, ':': -0.1494, ';': 0.5223, '<': 0.3736, '=': 0.2193, '>': -2.1398, '?': 3.2519, 'A': -0.3841, '[': 0.2734, '\\': -0.1707, ']': 0.5493, '^': 0.7976, '_': -0.6222, 'a': 0.0509, '{': 0.0975, '}': -0.1276}
-1.2
-3.1205 MOANA
-1.3805 Moana
-1.2761 I am Moana of Motunui.
2.7875 Do you want to build a snowman?
0.4846 I am a sussy programmer writing sussy code.
1.7608 What's wrong with you?
1.3641 Creates a criterion that measures the Binary Cross Entropy between the target and the input probabilities:
-1.3235 Table of Contents
-2.0935 https://harry7557558.github.io
-1.3672 1.1 4th-Order Partial Differential Equations
-1.0493 4th-Order Partial Differential Equations
0.6907 4th-order partial differential equations
-0.8504 From Wikipedia, the Free Encyclopedia.
0.0196 From Wikipedia, the free ency